## Randall Mencias
## Juan Francisco Cisneros
## Josue Cardenas

### DESCRIPCION DEL PROYECTO
- ....

#### Librerias

In [1]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

ModuleNotFoundError: No module named 'pandas'

### ANALISIS EXPLORATORIO DE DATOS

In [ ]:
# Where data is stored
pathTest = "../Medicine_Recomendation/data/test.csv"
pathTrain = "../Medicine_Recomendation/data/train.csv"

# Read data
dfTest = pd.read_csv(pathTest)
dfTrain = pd.read_csv(pathTrain)

# Size of the train and test data
print("Train data size: ", dfTrain.shape)
print("Test data size: ", dfTest.shape)

: 

In [ ]:
dfTrain.head()

: 

In [ ]:
dfTest.head()

: 

#### Plot Columns

In [ ]:
print("Test data shape, Rows and Columns: ", dfTest.shape)
print("Train data shape, Rows and Columns: ", dfTrain.shape)
print()
print("Test data columns: ", dfTest.columns)
print("Train data columns: ", dfTrain.columns)

: 

In [ ]:
dfTrain.info()

: 

In [ ]:
dfTest.info()

: 

In [ ]:
dfTrain.describe()

: 

In [ ]:
# Check for missing values
dfTrain.isnull().sum()

: 

In [ ]:
# Drop all rows with missing values
dfTrain.dropna(inplace=True)
dfTrain.isnull().sum()

: 

In [ ]:
dfTest.describe()

: 

In [ ]:
# Check for missing values
dfTest.isnull().sum()

: 

In [ ]:
print("Test data shape, Rows and Columns: ", dfTest.shape)
print("Train data shape, Rows and Columns: ", dfTrain.shape)

: 

In [ ]:
dfAll = pd.concat([dfTrain,dfTest])


: 

In [ ]:
condition_dn = dfAll.groupby(['condition'])['drugName'].nunique().sort_values(ascending=False)
condition_dn[0:20].plot(kind="bar", figsize = (14,6), fontsize = 10,color="green")
plt.xlabel("CONDITION", fontsize = 20)
plt.ylabel("DRUG COUNT", fontsize = 20)
plt.title("Top20 : The number of drugs per condition.", fontsize = 20)

: 

#### Drop all rows where the CONDITION is not in the top 5
- SINCE THE AMOUNT OF DATA IS TOO LARGE, WE WILL ONLY USE THE TOP 5 CONDITIONS, AND DROP THE REST OF THE DATA

In [ ]:
# Show all the conditions
dfAll['condition'].value_counts()

: 

In [ ]:
# Drop all rows not correponding to [Birth Control, Depresion, Pain, Anxiety and Acne]
dfAll = dfAll[dfAll['condition'].isin(['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne'])]
# Sort by condition
dfAll = dfAll.sort_values(by=['condition'])
dfAll

: 

#### Unique ID Drop
- SINCE THE ID IS UNIQUE FOR EACH PATIENT, WE WILL DROP IT, THERE IS NO USE FOR IT

In [ ]:
# Drop unique id
dfAll.drop(['uniqueID'], axis=1, inplace=True)
dfAll

: 

#### Lowercase all the data

In [ ]:
# Lowercase all text
dfAll['review'] = dfAll['review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dfAll['drugName'] = dfAll['drugName'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dfAll['condition'] = dfAll['condition'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dfAll

: 

#### Getting Rating Visualization

In [ ]:
ratings = dfAll.groupby(['rating'])['rating'].count()
ratings.plot(kind="bar", figsize = (14,6), fontsize = 10,color="green")
plt.xlabel("RATING", fontsize = 20)
plt.ylabel("COUNT", fontsize = 20)
plt.title("The number of ratings per rating.", fontsize = 20)

: 

#### Label Encoding

In [ ]:
# Label encoding for the condition
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dfAll['condition'] = le.fit_transform(dfAll['condition'])
dfAll

: 

In [ ]:
# print the unique values of the condition
print(dfAll['condition'].unique())

: 

In [ ]:
# Label encoding for the drugName
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dfAll['drugName'] = le.fit_transform(dfAll['drugName'])
dfAll

: 

In [ ]:
# print the unique values of the drugName
unique_drugName = dfAll['drugName'].unique()
#sort drugName
unique_drugName.sort()
print(unique_drugName)

: 

#### Dates Format

In [ ]:
dfAll['date'] = pd.to_datetime(dfAll['date'])

: 

##### The date will have more importance if is near to 2023

In [ ]:
dfAll['year'] = dfAll['date'].dt.year
# Get the minimum year and maximum year
print("Minimum year: ", dfAll['year'].min())
print("Maximum year: ", dfAll['year'].max())

: 

In [ ]:
# Knowing that the maximum year is 2017 and minimum is 2001, then give a rate from 0 to 1
# For each year
dfAll['year_importance'] = (dfAll['year'] - 2001) / (2017 - 2001)
dfAll

: 

#### Reviews

- WE NEED TO MAKE A SENTIMENT ANALYSIS OF THE REVIEWS
- SOME WORDS ARE NOT USEFUL FOR THE ANALYSIS, WE NEED TO CLEAN THE DATA
- OTHER WORDS ARE USEFUL FOR THE ANALYSIS, WE NEED TO KEEP THE DATA

In [ ]:
# take a look at five reviews 
dfAll['review']

: 

##### Lets delete special characters

In [ ]:
nltk.download('stopwords')

: 

In [ ]:
stops = set(stopwords.words('english'))

: 

In [ ]:
# Take in consideration this words
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    stops.remove(i)

: 

In [ ]:
stemmer = SnowballStemmer('english')

# Delete special characters and stopwords
def review_to_words(raw_review):
    # 1. Delete HTML 
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters
    words = letters_only.lower().split()
    # 5. Stopwords 
    meaningful_words = [w for w in words if not w in stops]
    # 6. Stemming
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. space join words
    return( ' '.join(stemming_words))

: 

In [ ]:
dfAll['review_clean'] = dfAll['review'].apply(review_to_words)

: 

In [ ]:
dfAll['review_clean']

: 

In [ ]:
# if a review has more than 500 characters, then take the first 500 characters
dfAll['review_clean'] = dfAll['review_clean'].apply(lambda x: x[:500])
dfAll['review'] = dfAll['review'].apply(lambda x: x[:500])


: 

#### Sentiment Analysis
- WE NEED TO MAKE A SENTIMENT ANALYSIS OF THE REVIEWS, TO ATTACK THE PROBLEM WE NEED TO KNOW IF THE REVIEW IS POSITIVE, NEUTRAL OR NEGATIVE
- AFTER A RESEARCH WE FOUND THAT THIS IS A GREAT WAY TO USE THE REVIEWS FURTHER IN THE ANALYSIS AND MODEL TRAINING

##### Making a Sentiment using each review rating
- WE ENCOUNTERD THAT IT IS BEST TO USE A PRE TRAINED MODEL TO MAKE THE SENTIMENT ANALYSIS

In [ ]:
save = True # IF THE SENTIMENT ANALYSIS LIBRARY IS NOT INSTALLED, SET THIS TO FALSE
sentimentAnalysisFinished = True

: 

In [ ]:
if not save:
    from transformers import AutoModelForSequenceClassification
    from transformers import TFAutoModelForSequenceClassification
    from transformers import AutoTokenizer
    import numpy as np
    from scipy.special import softmax
    import csv
    import urllib.request


    # Preprocess text (username and link placeholders)
    def preprocess(text):
        new_text = []
    
    
        for t in text.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)

    # Tasks:
    # emoji, emotion, hate, irony, offensive, sentiment
    # stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

    task='sentiment'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

    tokenizer = AutoTokenizer.from_pretrained(MODEL)



    # download label mapping
    labels=[]
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    labels = [row[1] for row in csvreader if len(row) > 1]

    # PT
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.save_pretrained(MODEL)

    text = "Good night 😊"
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # # TF
    # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
    # model.save_pretrained(MODEL)

    # text = "Good night 😊"
    # encoded_input = tokenizer(text, return_tensors='tf')
    # output = model(encoded_input)
    # scores = output[0][0].numpy()
    # scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

    saved = True


: 

In [ ]:
# use the sentiment analysis model to predict the sentiment of each review
def sentiment_score(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

: 

In [ ]:
# USING THE NOT REVIEW CLEAN
if not sentimentAnalysisFinished:
    # ALSO PRINT THE NUMBER OF REVIEW WHILE APLYING THE SENTIMENT ANALYSIS
    dfAll['sentiment_notClean'] = dfAll['review'].apply(sentiment_score)
    dfAll['sentiment_notClean'] = dfAll['sentiment_notClean'].apply(lambda x : 1 if np.argmax(x)==2 else -1 if np.argmax(x)==0 else 0)
    # save de df to a csv file
    dfAll.to_csv('dfAll_NotCleanReview.csv', index=False)
sentimentAnalysisFinished = True

: 

In [ ]:
dfAll

: 

In [ ]:
#load the df from the csv file
dfAll = pd.read_csv('dfAll_NotCleanReview.csv')
dfAll

: 

In [ ]:
sentimentAnalysisFinished = False
# USING THE CLEAN REVIEW
if not sentimentAnalysisFinished:
    dfAll['sentiment'] = dfAll['review_clean'].apply(sentiment_score)
    # Get the sentiment value, positive 1, negative -1, neutral 0
    dfAll['sentiment'] = dfAll['sentiment'].apply(lambda x : 1 if np.argmax(x)==2 else -1 if np.argmax(x)==0 else 0)
        # save de df to a csv file
    dfAll.to_csv('dfAll_CleanReview.csv', index=False)
sentimentAnalysisFinished = True



: 

In [ ]:
# load the df from the csv file
dfAll = pd.read_csv('dfAll_CleanReview.csv')
dfAll

: 